# BASELINE MODEL

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.dataset import Dataset
from src.model import LSTMmodel
from src.utils import *
from src.datamodule import DataModule
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.12.1-git20200711.33e2d80-dfsg1-0.6 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/mariano/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If 

In [56]:
# Get dataset 
data = read_clean_data('data/train_clinical_data.csv')
data = transform_dataframe_to_numpy(data)
#dataset = Dataset(data, window_size=10)
create_windows(data, 10, 4)[0]


(3, 2)
[array([[5.500e+02, 5.500e+01, 0.000e+00, 1.000e+01, 6.000e+00, 1.500e+01,
         1.900e+00, 0.000e+00],
        [5.530e+02, 5.500e+01, 3.000e+00, 1.000e+01, 7.000e+00, 2.500e+01,
         1.900e+00, 0.000e+00],
        [5.560e+02, 5.500e+01, 6.000e+00, 8.000e+00, 1.000e+01, 3.400e+01,
         1.900e+00, 0.000e+00],
        [5.590e+02, 5.500e+01, 9.000e+00, 8.000e+00, 9.000e+00, 3.000e+01,
         0.000e+00, 0.000e+00],
        [5.512e+03, 5.500e+01, 1.200e+01, 1.000e+01, 1.000e+01, 4.100e+01,
         0.000e+00, 0.000e+00],
        [5.518e+03, 5.500e+01, 1.800e+01, 7.000e+00, 1.300e+01, 3.800e+01,
         0.000e+00, 0.000e+00]], dtype=float32)
 array([[5.530e+02, 5.500e+01, 3.000e+00, 1.000e+01, 7.000e+00, 2.500e+01,
         1.900e+00, 0.000e+00],
        [5.560e+02, 5.500e+01, 6.000e+00, 8.000e+00, 1.000e+01, 3.400e+01,
         1.900e+00, 0.000e+00],
        [5.590e+02, 5.500e+01, 9.000e+00, 8.000e+00, 9.000e+00, 3.000e+01,
         0.000e+00, 0.000e+00],
        [5.512

/home/mariano/Desktop/parkinsonPrediction/src/utils.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  windows = np.array(windows)


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [51]:
# Create model 
model = LSTMmodel()

# Predict 
sample = dataset[0][0]
# Add batch dimension 
sample = sample.unsqueeze(0)
out = model(sample)
out.shape, out

(torch.Size([1, 4]),
 tensor([[-0.2012, -0.0588,  0.0737, -0.0447]], grad_fn=<AddmmBackward0>))

In [3]:
hparams = {
    'epochs': 10,
    'loss': 'mse',
    'patience': 5,
	'optimizer': 'Adam',
	'optimizer_params': {
		'lr': 1e-2
	},}
early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=hparams['patience'],
        verbose=False,
        mode='min'
    )
model = LSTMmodel(hparams)
dm = DataModule(32, 'data/train_clinical_data.csv')
trainer = pl.Trainer(max_epochs=hparams['epochs'], callbacks=[early_stopping])

# Train model
trainer.fit(model, dm)

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.12.1-git20200711.33e2d80-dfsg1-0.6 is an invalid version and will not be supported in a future release
  warnings.warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/mariano/.local/lib/python3.10/site-packages/pytorch_lig

In [59]:
# Get test data from datamodule
test_data, test_label = extract_test_data(dm)
test_data.shape, test_label.shape

((521, 10, 7), (521, 4))

In [55]:
pred_label = model.predict(test_data).numpy()
pred_label.shape

(521, 4)

In [57]:
# Smape error 
smape_metric(test_label, pred_label)

199.43212270736694